## Task 1:  Dependencies


In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

True

In [2]:
def check_if_env_var_is_set(env_var_name: str, human_readable_string: str = "API Key"):
    api_key = os.getenv(env_var_name)
  
    if api_key:
       print(f"{env_var_name} is present")
    else:
      print(f"{env_var_name} is NOT present, paste key at the prompt:")
      os.environ[env_var_name] = getpass.getpass(f"Please enter your {human_readable_string}: ")

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [3]:
import os
import getpass

check_if_env_var_is_set("OPENAI_API_KEY", "OpenAI API key")
check_if_env_var_is_set("TAVILY_API_KEY", "TAVILY API key")

OPENAI_API_KEY is present
TAVILY_API_KEY is present


In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - Certification Challenge"
check_if_env_var_is_set("LANGCHAIN_API_KEY", "LangSmith API Key")

LANGCHAIN_API_KEY is present


## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [5]:
from tavily import TavilyClient
import os
from typing import Optional


def tavily_studentaid_search(query: str) -> str:
    """
    Search ONLY StudentAid.gov for official federal information: FAFSA applications, 
    federal loan forgiveness programs, federal repayment plans, eligibility requirements.
    Use this when you need authoritative federal government information.
    
    Args:
        query: Search query for federal student aid topics
        
    Returns:
        Formatted search results from StudentAid.gov
    """
    try:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        
        response = client.search(
            query=f"site:studentaid.gov {query}",
            search_depth="advanced",
            max_results=3,
            include_answer=True,
            include_domains=["studentaid.gov"]
        )
        
        result = f"StudentAid.gov Search Results for: {query}\n\n"
        
        if response.get('answer'):
            result += f"Summary: {response['answer']}\n\n"
        
        result += "Official Federal Information:\n"
        for i, item in enumerate(response.get('results', []), 1):
            result += f"{i}. {item.get('title', 'No title')}\n"
            result += f"   {item.get('content', '')[:200]}...\n"
            result += f"   URL: {item.get('url', '')}\n\n"
        
        return result
        
    except Exception as e:
        return f"Error searching StudentAid.gov: {str(e)}"


def tavily_mohela_search(query: str) -> str:
    """
    Search ONLY Mohela loan servicer for account-specific help: making payments, 
    login issues, servicer-specific repayment options, customer service contacts.
    Use this when users have Mohela-serviced loans and need servicer-specific help.
    
    Args:
        query: Search query for Mohela servicer-specific information
        
    Returns:
        Formatted search results from Mohela
    """
    try:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        
        response = client.search(
            query=f"site:mohela.com OR site:servicing.mohela.com {query}",
            search_depth="advanced",
            max_results=3,
            include_answer=True,
            include_domains=["mohela.com", "servicing.mohela.com"]
        )
        
        result = f"Mohela Search Results for: {query}\n\n"
        
        if response.get('answer'):
            result += f"Summary: {response['answer']}\n\n"
        
        result += "Mohela Servicer Information:\n"
        for i, item in enumerate(response.get('results', []), 1):
            result += f"{i}. {item.get('title', 'No title')}\n"
            result += f"   {item.get('content', '')[:200]}...\n"
            result += f"   URL: {item.get('url', '')}\n\n"
        
        return result
        
    except Exception as e:
        return f"Error searching Mohela: {str(e)}"


def tavily_student_loan_search(query: str, source: Optional[str] = None) -> str:
    """
    Compare information across BOTH federal sources and Mohela when user needs 
    comprehensive view or comparison of student loan options. Use this when users 
    want to see both federal policies and servicer-specific implementation, or when 
    they're unsure which source has the information they need.
    
    Args:
        query: Search query for student loan information requiring comparison
        source: Optional - "studentaid" for StudentAid.gov only, "mohela" for Mohela only
        
    Returns:
        Formatted search results comparing both sources
    """
    try:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        
        if source == "studentaid":
            return tavily_studentaid_search(query)
        elif source == "mohela":
            return tavily_mohela_search(query)
        else:
            # Search both sources for comparison
            search_query = f"student loan {query} site:studentaid.gov OR site:mohela.com"
            
            response = client.search(
                query=search_query,
                search_depth="advanced",
                max_results=5,
                include_answer=True,
                include_domains=["studentaid.gov", "mohela.com", "servicing.mohela.com"]
            )
            
            result = f"Comprehensive Student Loan Search Results for: {query}\n\n"
            
            if response.get('answer'):
                result += f"Overall Summary: {response['answer']}\n\n"
            
            # Group results by domain for comparison
            studentaid_results = []
            mohela_results = []
            
            for item in response.get('results', []):
                url = item.get('url', '')
                if 'studentaid.gov' in url:
                    studentaid_results.append(item)
                elif 'mohela.com' in url:
                    mohela_results.append(item)
            
            if studentaid_results:
                result += "📋 Federal Government Perspective (StudentAid.gov):\n"
                for i, item in enumerate(studentaid_results[:2], 1):
                    result += f"{i}. {item.get('title', 'No title')}\n"
                    result += f"   {item.get('content', '')[:150]}...\n"
                    result += f"   Source: {item.get('url', '')}\n\n"
            
            if mohela_results:
                result += "🏢 Loan Servicer Perspective (Mohela):\n"
                for i, item in enumerate(mohela_results[:2], 1):
                    result += f"{i}. {item.get('title', 'No title')}\n"
                    result += f"   {item.get('content', '')[:150]}...\n"
                    result += f"   Source: {item.get('url', '')}\n\n"
            
            if not studentaid_results and not mohela_results:
                result += "No specific results found from StudentAid.gov or Mohela. Consider using general web search.\n"
            
            return result
            
    except Exception as e:
        return f"Error searching student loan information: {str(e)}"


In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

from langchain.tools import Tool

tavily_tool = TavilySearchResults(max_results=5)

# tool_belt = [
#     tavily_tool,
#     ArxivQueryRun(),
# ]

tool_belt = [
    tavily_tool,
    Tool(
        name="StudentAid_Federal_Search",
        description="Search ONLY StudentAid.gov for official federal information: FAFSA applications, federal loan forgiveness programs, federal repayment plans, eligibility requirements",
        func=tavily_studentaid_search
    ),
    Tool(
        name="Mohela_Servicer_Search", 
        description="Search ONLY Mohela loan servicer for account-specific help: making payments, login issues, servicer-specific repayment options, customer service contacts",
        func=tavily_mohela_search
    ),
    Tool(
        name="Student_Loan_Comparison_Search",
        description="Compare information across BOTH federal sources and Mohela when user needs comprehensive view or comparison of student loan options",
        func=tavily_student_loan_search
    ),
    ArxivQueryRun(),
]

/tmp/ipykernel_3124/679774791.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [8]:
model = model.bind_tools(tool_belt)

In [9]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

In [10]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [11]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [12]:
uncompiled_graph.set_entry_point("agent")

In [13]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

In [14]:
uncompiled_graph.add_edge("action", "agent")

In [15]:
simple_agent_graph = uncompiled_graph.compile()

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [16]:
import importlib
import tool_calls_parser
importlib.reload(tool_calls_parser)
from tool_calls_parser import parse_logs, print_formatted_results

In [17]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
          print_formatted_results(parse_logs(str(values["messages"])))
        print("\n")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'


[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8frtUcW0lq993ybwnysEfS54', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 310, 'total_tokens': 333, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-Bzaw7dpmYekX32fa8yu0Da2sxSakN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e756eedf-fb70-4cba-b80d-c0f45d6dd450-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_8frtUcW0lq993ybwnysEfS

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [18]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
          print_formatted_results(parse_logs(str(values["messages"])))
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qWCsMXce5OBaDgRwDfRWOJ8N', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_iBNDr3tUBq9qzJwO1Qd4qsd6', 'function': {'arguments': '{"query": "latest Tweet of each author of the QLoRA paper"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 326, 'total_tokens': 386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-BzawDGRfjgfJ5dONfvMpErVGhKQGa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c2f9fe08-d9b1-4123-a1f8-5353c0007f3f-0', tool_cal

In [19]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is the issue with Aidvantage in the borrower's complaint?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
          print_formatted_results(parse_logs(str(values["messages"])))
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nERYrnrUONQ7mP4q02u5anD6', 'function': {'arguments': '{"__arg1":"Aidvantage borrower complaint"}', 'name': 'StudentAid_Federal_Search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 314, 'total_tokens': 337, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-BzawLOqSQoKXgfKMZaYBh9LvXqMZS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4b064091-0a2c-4bc4-8146-ca9974ffd855-0', tool_calls=[{'name': 'StudentAid_Federal_Search', 'args': {'__arg1': 'Aidvantage borrower complaint'}, 'id': 'call_nERYrnrUONQ7mP4q02u5anD6', 'type': 'to

In [20]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What concerns does the borrower have regarding Nelnet's communication about their student loan issuer?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
          print_formatted_results(parse_logs(str(values["messages"])))
        print("\n")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'


[AIMessage(content="To provide an accurate answer, I need to know the specific concerns the borrower has expressed about Nelnet's communication regarding their student loan issuer. Could you please provide more details or specify the concerns?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 317, 'total_tokens': 357, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-BzawPsaKdf4F5V0eCN0BX1FN1EOYD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--02e4ccaf-df57-4667-878d-1451b9f18202-0', usage_metadata={'input_tokens': 317, 'output_tokens': 40, 'total_tokens': 357, 'input_token_details': {'audio': 0, 'cache

In [43]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="How does NelNet's repeated failure to process auto-debit payments, despite confirmation, relate to borrower rights and potential violations of privacy laws?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
          # print_formatted_results(parse_logs(str(values["messages"])))
          parsed_data = parse_langchain_messages(values["messages"])
          print_formatted_results(parsed_data)
        print("\n")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'


[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jZbSbFOH0gGCw5Laa8pMYnFt', 'function': {'arguments': '{"__arg1": "NelNet auto-debit payment issues"}', 'name': 'StudentAid_Federal_Search'}, 'type': 'function'}, {'id': 'call_02JtAJ0KsUzgPrBbtd9lI5Cj', 'function': {'arguments': '{"__arg1": "borrower rights auto-debit payments"}', 'name': 'StudentAid_Federal_Search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 328, 'total_tokens': 394, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-Bzb4uraB30ZLJZCLCVLoRrJd8X5T4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f1387126-d

In [22]:
inputs = {"messages" : [HumanMessage(content="How does NelNet's repeated failure to process auto-debit payments, despite confirmation, relate to borrower rights and potential violations of privacy laws?")]}
response = simple_agent_graph.invoke(inputs)

In [40]:
import importlib
import tool_calls_parser_for_eval
importlib.reload(tool_calls_parser_for_eval)
from tool_calls_parser_for_eval import parse_tool_call, parse_langchain_messages, extract_contexts_for_eval, print_formatted_results
# from tool_calls_parser import parse_langchain_messages, print_formatted_results, extract_contexts_for_eval

evaluation_contexts = extract_contexts_for_eval(response["messages"])
print(f"✅ Extracted {len(evaluation_contexts)} contexts for evaluation")

✅ Extracted 10 contexts for evaluation


In [41]:
parsed_data = parse_langchain_messages(response["messages"])
print_formatted_results(parsed_data)

=== PARSING RESULTS ===
Tool calls: 2
Tool results: 4
Tools used: [None, 'Mohela_Servicer_Search', 'StudentAid_Federal_Search']

=== TOOL CALLS ===
1. StudentAid_Federal_Search -> {'__arg1': 'NelNet auto-debit payment issues borrower rights privacy laws'}
2. Mohela_Servicer_Search -> {'__arg1': 'NelNet auto-debit payment issues borrower rights privacy laws'}

=== TOOL RESULTS ===
1. None -> How does NelNet's repeated failure to process auto-debit payments, despite confirmation, relate to b...
2. StudentAid_Federal_Search -> StudentAid.gov Search Results for: NelNet auto-debit payment issues borrower rights privacy laws  Su...
3. Mohela_Servicer_Search -> Mohela Search Results for: NelNet auto-debit payment issues borrower rights privacy laws  Summary: M...
4. None -> NelNet handles auto-debit payments for federal student loans and is committed to protecting borrower...

=== CONTEXT EXTRACTION DEMO ===
Extracted 10 contexts for evaluation:
1. StudentAid.gov Search Results for: NelNet au

In [42]:
eval_sample = {
    "user_input": inputs["messages"][0].content,
    "response": response["messages"][-1].content,  # Final AI response
    "retrieved_contexts": evaluation_contexts,
    "tools_used": parsed_data['summary']['tools'],
    "num_contexts": len(evaluation_contexts)
}

print(f"\n🎯 EVALUATION SAMPLE:")
print(f"Query: {eval_sample['user_input']}")
print(f"Response: {eval_sample['response'][:200]}...")
print(f"Contexts: {eval_sample['num_contexts']} extracted")
print(f"Tools: {eval_sample['tools_used']}")


🎯 EVALUATION SAMPLE:
Query: How does NelNet's repeated failure to process auto-debit payments, despite confirmation, relate to borrower rights and potential violations of privacy laws?
Response: NelNet handles auto-debit payments for federal student loans and is committed to protecting borrower rights and privacy laws. According to official federal information, NelNet provides FAQs and resour...
Contexts: 10 extracted
Tools: [None, 'Mohela_Servicer_Search', 'StudentAid_Federal_Search']
